# Once we set our hyperparameters and features we can test on our validation set and see how our model performs 

In [24]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
import logging

from sklearn.preprocessing import PolynomialFeatures, normalize, StandardScaler, MinMaxScaler
from sklearn.linear_model import SGDRegressor, LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor, AdaBoostRegressor

from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
pd.set_option('display.max_columns', 300)
pd.set_option('display.max_rows', 300)

In [25]:
train_data = pd.read_csv('../Data Scraping/DataTrainSet.csv')
test_data  = pd.read_csv('../Data Scraping/Cut_Data/data_2019_Norm.csv')


train_data = train_data.drop(train_data[train_data.points_won <0.01].index)
test_data = test_data.drop(test_data[test_data.pts_per_g <0.6].index)

features = ['ts_pct', 'bpm', 'pts_per_g', 'trb_per_g', 'ast_per_g',
            'stl_per_g', 'blk_per_g', 'ws'];

features2 = ['player','ts_pct', 'bpm', 'pts_per_g', 'trb_per_g', 'ast_per_g',
            'stl_per_g', 'blk_per_g', 'ws', 'points_won'];

In [26]:
train_data.columns

Index(['player', 'pos_x', 'age_x', 'team_id_x', 'g_x', 'mp_x', 'fg_per_poss',
       'fga_per_poss', 'fg_pct_x', 'fg3_per_poss', 'fg3a_per_poss',
       'fg3_pct_x', 'fg2_per_poss', 'fg2a_per_poss', 'fg2_pct_x',
       'ft_per_poss', 'fta_per_poss', 'ft_pct_x', 'orb_per_poss',
       'drb_per_poss', 'trb_per_poss', 'ast_per_poss', 'stl_per_poss',
       'blk_per_poss', 'tov_per_poss', 'pf_per_poss', 'pts_per_poss',
       'off_rtg', 'def_rtg', 'per', 'ts_pct', 'fg3a_per_fga_pct',
       'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct',
       'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'ws-dum', 'ows', 'dws',
       'ws', 'ws_per_48', 'bpm-dum', 'obpm', 'dbpm', 'bpm', 'vorp', 'mp_per_g',
       'fg_per_g', 'fga_per_g', 'fg3_per_g', 'fg3a_per_g', 'fg2_per_g',
       'fg2a_per_g', 'efg_pct', 'ft_per_g', 'fta_per_g', 'orb_per_g',
       'drb_per_g', 'trb_per_g', 'ast_per_g', 'stl_per_g', 'blk_per_g',
       'tov_per_g', 'pf_per_g', 'pts_per_g', 'fg_per_mp', 'fga_per_mp',
    

In [31]:
resultsA = np.zeros((25,3)); 
def get_predictions_for_regression_model(model, poly_fit, train_data, test_data,  year, should_print=True):
    test_data  = pd.read_csv('../Data Scraping/Cut_Data/data_'+str(year)+'_Norm.csv')
    test_data = test_data.drop(test_data[test_data.pts_per_g <0.6].index)
    
    
    train_x = train_data[features].to_numpy()
    train_y = train_data['points_won'].to_numpy()
    
    train_y = train_y.reshape(train_y.shape[0], )
    
    test_x = test_data[features].to_numpy()
    test_x = np.nan_to_num(test_x)
    test_data2 = test_data[features2].to_numpy()
    
    if poly_fit is not None:
        train_x = poly_fit.fit_transform(train_x)
        test_x = poly_fit.fit_transform(test_x)
    
    model.fit(train_x, train_y)
    predict_y = model.predict(test_x)
    predict_y = predict_y.reshape(predict_y.shape[0], 1)
    
    
    results = np.append(test_data2, predict_y[:], 1)
    return results

In [32]:
yearList = [2018]
#predictionArray
for i in yearList:
    A1=get_predictions_for_regression_model(
        model = RandomForestRegressor(n_estimators=100, min_samples_split=2, oob_score=True),
        poly_fit = PolynomialFeatures(degree=1, interaction_only=False),
        train_data=train_data,
        test_data=test_data, 
        year = i
        
    );
        

for i in yearList:
    A2 = get_predictions_for_regression_model(
        model = Ridge(alpha=1.0, solver = 'svd'),
        poly_fit = PolynomialFeatures(degree=2, interaction_only=True),
        train_data=train_data,
        test_data=test_data, 
        year = i
    )

    
for i in yearList:
    A3 = get_predictions_for_regression_model(
        model = GradientBoostingRegressor(  loss='ls', learning_rate=0.5, n_estimators=300),
        poly_fit = PolynomialFeatures(degree=2, interaction_only=True),
        train_data=train_data,
        test_data=test_data, 
        year = i
    )
    
for i in yearList:
    A4 = get_predictions_for_regression_model(
        model = SGDRegressor( loss='squared_epsilon_insensitive',
            epsilon=0.0001,
            learning_rate='adaptive',
            eta0=0.01),
        poly_fit = PolynomialFeatures(degree=2, interaction_only=True),
        train_data=train_data,
        test_data=test_data, 
        year = i
    )
for i in yearList:
    A5 = get_predictions_for_regression_model(
        model = MLPRegressor(hidden_layer_sizes=(1000,), activation='relu',solver='lbfgs'),
        poly_fit = PolynomialFeatures(degree=2, interaction_only=True),
        train_data=train_data,
        test_data=test_data, 
        year = i
    )


/Users/maddocbf/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDRegressor in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


In [33]:
score = np.zeros(shape=(0,0))
for i in range(0, A1.shape[0]):
    score = np.append(score,A1[i,-1]+A2[i,-1]+A3[i,-1]+A4[i,-1]+A5[i,-1])

names = np.zeros(shape=(0,0))
for i in range(0, A1.shape[0]):
    names = np.append(names, A2[i,0])
    
scores = np.zeros(shape=(A1.shape[0], 3))
data = {"Player Name":names, "Random Forest":A1[:,-1],"Ridge":A2[:,-1],"Gradient Boosting":A3[:,-1],"SGD":A4[:,-1],"MLP":A5[:,-1],"Actual Score":A1[:,-2],"score":score}
scoresDF = pd.DataFrame(data)

In [34]:
print(scoresDF.sort_values('score'))

              Player Name Random Forest       Ridge Gradient Boosting  \
29        Dennis Schröder     0.0502706   -0.141339           0.04533   
5          MarShon Brooks      0.036411   -0.292486        0.00412804   
33              John Wall      0.129404   -0.130593         -0.159979   
4            Devin Booker     0.0489842    -0.13106         0.0246029   
13           Tyreke Evans     0.0445656  -0.0944478         0.0474115   
25       Donovan Mitchell     0.0280649  -0.0901867          0.037526   
18           Jrue Holiday     0.0377613   0.0178136        0.00409351   
24        Khris Middleton     0.0285163  -0.0153688         0.0364625   
34            T.J. Warren     0.0381488    -0.18536         0.0193434   
2         Harrison Barnes     0.0313144   -0.170454          0.129883   
36           Lou Williams     0.0492182   0.0149874         0.0636348   
17          Tobias Harris     0.0330138  -0.0622816         0.0378527   
30          Klay Thompson     0.0368474   -0.117832